### Imports

In [21]:
import pandas as pd

url1 = "https://raw.githubusercontent.com/JoseAPBarbosa/projeto-maquinas-termicas/main/saturated_by_temperature_V1.5.csv"
url2 = "https://raw.githubusercontent.com/JoseAPBarbosa/projeto-maquinas-termicas/main/saturated_by_pressure_V1.4.csv"
url3 = "https://raw.githubusercontent.com/JoseAPBarbosa/projeto-maquinas-termicas/main/compressed_liquid_and_superheated_steam_V1.3.csv"

sat_temperature_df = pd.read_csv(url1)
sat_temperature_df.rename(columns={
    'T (°C)': 'T',
    'P (MPa)': 'P_sat',
    'Specific Volume Liquid (m^3/kg)': 'v_f',
    'Specific Volume Vapor (m^3/kg)': 'v_g',
    'Internal Energy Liquid (kJ/kg)': 'u_f',
    'Internal Energy Vapor (kJ/kg)': 'u_g',
    'Internal Energy of Vaporization (kJ/kg)': 'u_fg',
    'Enthalpy Liquid (kJ/kg)': 'h_f',
    'Enthalpy Vapor (kJ/kg)': 'h_g',
    'Enthalpy of Vaporization (kJ/kg)': 'h_fg',
    'Entropy Liquid [kJ/(kg K)]': 's_f',
    'Entropy Vapor [kJ/(kg K)]': 's_g',
    'Entropy of Vaporization [kJ/(kg K)]': 's_fg'
}, inplace=True)

sat_pressure_df = pd.read_csv(url2)
sat_pressure_df.rename(columns={
    'P (MPa)': 'P',
    'T (°C)': 'T_sat',
    'Specific Volume Liquid (m^3/kg)': 'v_f',
    'Specific Volume Vapor (m^3/kg)': 'v_g',
    'Internal Energy Liquid (kJ/kg)': 'u_f',
    'Internal Energy Vapor (kJ/kg)': 'u_g',
    'Internal Energy of Vaporization (kJ/kg)': 'u_fg',
    'Enthalpy Liquid (kJ/kg)': 'h_f',
    'Enthalpy Vapor (kJ/kg)': 'h_g',
    'Enthalpy of Vaporization (kJ/kg)': 'h_fg',
    'Entropy Liquid [kJ/(kg K)]': 's_f',
    'Entropy Vapor [kJ/(kg K)]': 's_g',
    'Entropy of Vaporization [kJ/(kg K)]': 's_fg'
}, inplace=True)

cliquid_ssteam_df = pd.read_csv(url3)
cliquid_ssteam_df.rename(columns={
    'Pressure (MPa)': 'P',
    ' Temperature (°C)': 'T',
    ' Specific Volume (m^3/kg)': 'v',
    ' Density (kg/m^3)': 'rho',
    ' Specific Internal Energy (kJ/kg)': 'u',
    ' Specific Enthalpy (kJ/kg)': 'h',
    ' Specific Entropy [kJ/(kg K)]': 's'
}, inplace=True)
cliquid_ssteam_df = cliquid_ssteam_df.apply(pd.to_numeric, errors='coerce')

### Função para obter o estado termodinâmico da água para um dada pressão e propriedade

In [22]:
def interpolated_state_at_P(df, target_P, col, target_val):
    """
    Interpolates all values in df at fixed target_P so that column `col`
    equals target_val exactly (linear interpolation).
    
    Parameters:
        df : pandas.DataFrame
        target_P : float
            Desired pressure value
        col : str
            Column name to interpolate against (e.g. 'h', 's', etc.)
        target_val : float
            Desired value in column `col`
    
    Returns:
        interpolated_row : pandas.Series
            Row with interpolated values
    """
    # Filter only rows at given pressure
    df_at_P = df[df['P'] == target_P].sort_values(by=col).reset_index(drop=True)
    
    # If the target_val is outside range, return closest row
    if target_val <= df_at_P[col].min():
        return df_at_P.iloc[0]
    if target_val >= df_at_P[col].max():
        return df_at_P.iloc[-1]

    # Find bounding rows
    lower_idx = (df_at_P[col] <= target_val).idxmax()  # first >=
    upper_idx = (df_at_P[col] >= target_val).idxmax()  # first >= again

    # Handle possible edge-case
    if lower_idx == upper_idx:
        return df_at_P.loc[lower_idx]

    row1 = df_at_P.loc[lower_idx]
    row2 = df_at_P.loc[upper_idx]

    # Linear interpolation factor
    t = (target_val - row1[col]) / (row2[col] - row1[col])

    # Interpolate every column numerically
    interpolated = {}
    for c in df.columns:
        if c == 'P':
            interpolated[c] = target_P
        elif pd.api.types.is_numeric_dtype(df[c]):
            interpolated[c] = row1[c] + t * (row2[c] - row1[c])
        else:
            # Non-numeric (like phase labels) – keep closest
            interpolated[c] = row1[c] if t < 0.5 else row2[c]

    return pd.Series(interpolated)


---

### Turbina à gas

In [23]:
Wgas = -5.05e3
m20 = 19.5
T20 = 545.0
hr = 11914

Mf =  0.8924*16 + 0.0786*30 + 0.0024*44 + 0.0005*58 + 0.01134*28 + 0.0125*44 + 0.0002*32
rho = 0.78
LHV = 35564

Mair = 0.21*32 + 0.79*28

In [24]:
R = 8.314
cp_co2 = lambda T: (R/44) * ((+2.401) + (+8.735e-3)*T + (-6.607e-6)*T**2 + (+2.002e-9)*T**3 +            0*T**4)
cp_h2o = lambda T: (R/18) * ((+4.070) + (-1.108e-3)*T + (+4.152e-6)*T**2 + (-2.964e-9)*T**3 + (+0.807e-12)*T**4)
cp_o2  = lambda T: (R/32) * ((+3.626) + (-1.878e-3)*T + (+7.055e-6)*T**2 + (-6.764e-9)*T**3 + (+2.156e-12)*T**4)
cp_n2  = lambda T: (R/28) * ((+3.675) + (-1.208e-3)*T + (+2.324e-6)*T**2 + (-0.632e-9)*T**3 + (-0.226e-12)*T**4)

cp_CO2 = cp_co2(800)
cp_H2O = cp_h2o(800)
cp_O2  = cp_o2(800)
cp_N2  = cp_n2(800)

In [25]:
q_in = -Wgas * hr/3600
mf = q_in/LHV * rho
mair = m20 - mf

nair = mair/Mair
nf = mf/Mf

a = 0.8924 + 2*0.0786 + 3*0.0024 + 4*0.0005 + 0.0125
b = (4*0.8924 + 6*0.0786 + 8*0.0024 + 10*0.0005)/2
c = (2*0.0125 + 2*0.0002 + 2*(nair/nf)*0.21 - 2*a - b)/2
d = (2*0.0134 + 2*(nair/nf)*0.79)/2

cp_out = (a*cp_CO2 + b*cp_H2O + c*cp_O2 + d*cp_N2)/(a + b + c + d)

In [26]:
mco2 = nf*a*44

---

### Entrada:

In [27]:
P1 = 5.0
T1 = 240.0

m1 = 80.0

---

### Quedas de pressão

In [28]:
P2 = 1.0
P6 = 0.2
P7 = 0.2
P12 = 0.05

### Temperaturas pós-aquecimento

In [29]:
T5 = 300
T10 = 250

---

### Primeiro Flashing

In [30]:
state_1 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P1) & (cliquid_ssteam_df["T"] == T1)]
h1 = state_1["h"].values[0]
s1 = state_1["s"].values[0]
x1 = 0.0

state_2 = sat_pressure_df[sat_pressure_df["P"] == P2]
T2 = state_2["T_sat"].values[0]
hf2 = state_2["h_f"].values[0]
hg2 = state_2["h_g"].values[0]
sf2 = state_2["s_f"].values[0]
sg2 = state_2["s_g"].values[0]

x2 = (h1 - hf2)/(hg2 - hf2)
h2 = x2*hg2 + (1-x2)*hf2
s2 = x2*sg2 + (1-x2)*sf2
m2 = m1

P3 = P2
T3 = T2
h3 = hf2
s3 = sf2
x3 = 0.0
m3 = (1-x2)*m2

P4 = P2
T4 = T2
h4 = hg2
s4 = sg2
x4 = 1.0
m4 = x2*m2

---

### Segundo Flashing

In [31]:
state_7 = sat_pressure_df[sat_pressure_df["P"] == P7]
T7 = state_7["T_sat"].values[0]
hf7 = state_7["h_f"].values[0]
hg7 = state_7["h_g"].values[0]
sf7 = state_7["s_f"].values[0]
sg7 = state_7["s_g"].values[0]

x7 = (h3 - hf7)/(hg7 - hf7)
h7 = x7*hg7 + (1-x7)*hf7
s7 = x7*sg7 + (1-x7)*sf7
m7 = m3

P8 = P7
T8 = T7
h8 = hf7
s8 = sf7
x8 = 0.0
m8 = (1-x7)*m7

P9 = P7
T9 = T7
h9 = hg7
s9 = sg7
x9 = 1.0
m9 = x7*m7

---

### Trocador de calor 1

In [32]:
P5 = P4
state_5 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P5) & (cliquid_ssteam_df["T"] == T5)]
h5 = state_5["h"].values[0]
s5 = state_5["s"].values[0]
x5 = x4
m5 = m4

T21 = T20 - (h5 - h4)/(m20*cp_out)*m5

### Trocador de calor 2

In [33]:
P10 = P9
state_10 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P10) & (cliquid_ssteam_df["T"] == T10)]
h10 = state_10["h"].values[0]
s10 = state_10["s"].values[0]
x10 = x9
m10 = m9

m21 = m20
T22 = T21 - (h10 - h9)/(m21*cp_out)*m10

### Emissões de carbono [Ton/h]

---

### Turbina de alta pressão


In [34]:
eta_s = 0.85
s6 = s5

state_6_s = interpolated_state_at_P(cliquid_ssteam_df, P6, "s", s6)
h6s = state_6_s["h"]
h6 = h5 - eta_s*(h5 - h6s)

state_6 = interpolated_state_at_P(cliquid_ssteam_df, P6, "h", h6)
T6 = state_6["T"]
s6 = state_6["s"]
x6 = x5
m6 = m5

wt1 = h6 - h5

---

### Misturador 6-10-11

In [35]:
P11 = P10
m11 = m6 + m10
h11 = (m6*h6 + m10*h10)/m11

state_11 = interpolated_state_at_P(cliquid_ssteam_df, P11, "h", h11)
T11 = state_11["T"]
s11 = state_11["s"]
x11 = x10

---

### Turbina de baixa pressão

In [36]:
s12 = s11

state_12_s = interpolated_state_at_P(cliquid_ssteam_df, P12, "s", s12)
h12s = state_12_s["h"]
h12 = h11 - eta_s*(h11 - h12s)

state_12 = interpolated_state_at_P(cliquid_ssteam_df, P12, "h", h12)
T12 = state_6["T"]
s12 = state_6["s"]
x12 = x11
m12 = m11

wt2 = h12 - h11

---

### Compressor 1

In [37]:
T13 = 25
P13 = P12
state_13 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P13) & (cliquid_ssteam_df["T"] == T13)]
v13 = state_13['v'].values[0]
h13 = state_13['h'].values[0]
s13 = state_13['s'].values[0]
x13 = 0.0
m13 = m12

P14 = 0.2
wb1 = v13*(P14*10**6 - P13*10**6)*10**-3

h14 = h13 + wb1
state_14 = interpolated_state_at_P(cliquid_ssteam_df, P14, "h", h14)
T14 = state_14['T']
s14 = state_14['s']
x14 = x13
m14 = m13

### Misturador 8-14-15

In [38]:
P15 = P14
m15 = m8 + m14
h15 = (m8*h8 + m14*h14)/m15
state_15 = interpolated_state_at_P(cliquid_ssteam_df, P15, "h", h15)
T15 = state_15["T"]
v15 = state_15['v']
s15 = state_15["s"]
x15 = x14

### Compressor 2

In [39]:
P16 = 1.0
wb2 = v15*(P16*10**6 - P15*10**6)*10**-3

h16 = h15 + wb2
state_16 = interpolated_state_at_P(cliquid_ssteam_df, P16, "h", h16)
T16 = state_16['T']
s16 = state_16['s']
x16 = 0.0
m16 = m15

---

### Trabalho Total

In [40]:
Wt = m5*wt1 + m11*wt2
Wb = m13*wb1 + m15*wb2
Wv = Wt + Wb
W = Wv + Wgas
Ql = m12*(h12 - h13)

print(
    f"Entrada:\n"
    f"\tPoço geotérmico:\t\t{m1:.1f} kg/s\n"
    f"\tGás natural:\t\t\t{mf:.2f} kg/s\n\n"
    f"Saída:\n"
    f"\tPotência gerada:\n"
    f"\t\tGeotermica: \t\t {-Wv/1000:.3f} MW\t {Wv/W*100:.1f}%\n"
    f"\t\tTurbina a gás: \t\t {-Wgas/1000:.3f} MW\t {Wgas/W*100:.1f}%\n"
    f"\t\tTotal: \t\t\t{-W/1000:.3f} MW\t{W/W*100:.1f}%\n"
    f"\tCalor liberado: \t\t{Ql/1000:.3f} MW\n\n"
    f"Emissões de CO2:\n"
    f"\tTonelada por megawatt hora: \t {mco2*3.6/(-W/1000):.3f} Ton/MWh\n\n"
    )

Entrada:
	Poço geotérmico:		80.0 kg/s
	Gás natural:			0.37 kg/s

Saída:
	Potência gerada:
		Geotermica: 		 7.184 MW	 58.7%
		Turbina a gás: 		 5.050 MW	 41.3%
		Total: 			12.234 MW	100.0%
	Calor liberado: 		48.133 MW

Emissões de CO2:
	Tonelada por megawatt hora: 	 0.288 Ton/MWh


